In [1]:
%load_ext autoreload
%autoreload 2

In [7]:
import torch.nn as nn
import torch
import pytorch_lightning as pl
from bayesian_nnet.net_utils import initialize_pl_trainer
from bayesian_nnet.data import IQDataModel
from bayesian_nnet.network import AutoModClassifier
from bayesian_nnet.data import initialize_data_splits

### Initialize data split indexing

In [3]:
out_of_distrib_modes = ["lsb",
                        "usb"]

initialize_data_splits(out_of_distrib_modes)

In [ ]:
iqdata = IQDataModel()
model = AutoModClassifier()
callbacks, mlf_logger = initialize_pl_trainer("frequentist_net")

trainer = pl.Trainer(callbacks=callbacks,
                     max_epochs=10,
                     logger=mlf_logger)

trainer.fit(model, iqdata)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type                  | Params | Mode 
-----------------------------------------------------------------
0 | iq_embed       | IQDataEmbedder        | 46.5 K | train
1 | modclf         | FrequentistClassifier | 2.1 M  | train
2 | loss           | CrossEntropyLoss      | 0      | train
3 | train_accuracy | MulticlassAccuracy    | 0      | train
4 | val_accuracy   | MulticlassAccuracy    | 0      | train
5 | test_accuracy  | MulticlassAccuracy    | 0      | train
6 | softmax        | Softmax               | 0      | train
-----------------------------------------------------------------
2.1 M     Trainable params
0         Non-trainable params
2.1 M     Total params
8.596     Total estimated model params size (MB)
29        Modules in train mode
0         Modules in eval mode


Sanity Checking: |                                                                                  | 0/? [00:…

Training: |                                                                                         | 0/? [00:…

Validation: |                                                                                       | 0/? [00:…